In [2]:
import pytube
import pandas as pd
import moviepy.editor as mp
import math
import os
import speech_recognition as sr
from pydub import AudioSegment
from pydub.utils import make_chunks 
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

## Downloading Video

In [3]:
link = "https://www.youtube.com/watch?v=MnT1xgZgkpk&t=4s"
yt = pytube.YouTube(link)
path = "C:\\Users\\PRAVEEN-PP\\OneDrive\\Desktop\\video_text\\videos"
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').first().download(path)

'C:\\Users\\eswar\\Desktop\\vscode\\video_text\\videos\\What happens when our computers get smarter than we are  Nick Bostrom.mp4'

In [6]:
clip = mp.VideoFileClip("C:\\Users\\PRAVEEN-PP\\OneDrive\\Desktop\\video_text\\videos\\What happens when our computers get smarter than we are  Nick Bostrom.mp4")
clip.audio.write_audiofile("C:\\Users\\PRAVEEN-PP\\OneDrive\\Desktop\\video_text\\videos\\test.wav",codec='pcm_s16le')

MoviePy - Writing audio in C:\Users\eswar\Desktop\vscode\video_text\videos\test.wav


MoviePy - Done.


In [7]:
def generateText(path):
    print("1")
    audio = AudioSegment.from_file(path)
    segment_duration = 10000
    start_time = 0
    r = sr.Recognizer()
    l=[]
    while start_time < len(audio):
        end_time = start_time + segment_duration
        if end_time > len(audio):
            end_time = len(audio)
        segment = audio[start_time:end_time]
        segment.export("temp_audio_file.wav", format="wav")
        
        segment_duration = 10000
        with sr.AudioFile("temp_audio_file.wav") as source:
               audio_data = r.listen(source)
               try:
                      text = r.recognize_google(audio_data)
                      l.append(text)
                      print("Segment from {} to {}: {}".format(start_time, end_time, text))
               except:
                      print("not recognized")
               start_time = end_time
               text='.'.join(l)
               print(text)
    return text

## Getting Final Text from Video

In [10]:
path = "C:\\Users\\PRAVEEN-PP\\OneDrive\\Desktop\\video_text\\videos\\test.wav"
final_text = generateText(path)

1
result2:
[]
not recognized

result2:
{   'alternative': [   {   'confidence': 0.88687527,
                           'transcript': 'I work with a bunch of mathematician '
                                         'classification'},
                       {   'transcript': 'I work with a bunch of mathematician '
                                         'classification Computer Sciences'},
                       {   'transcript': 'I work with a bunch of mathematician '
                                         'classification Computer Sciences and '
                                         'then think'},
                       {   'transcript': 'I work with a bunch of mathematician '
                                         'classification Computer Sciences and '
                                         'visit around the sync'},
                       {   'transcript': 'I work with a bunch of mathematician '
                                         'classification Computer Sciences and '

In [11]:
print(final_text)

I work with a bunch of mathematician classification.some people think that something.exercise okay let's look at the modern human condition this is the normal way for.Pathan you recently ride guess on this planet the human species well like.if the world like was created or first grade in 1 year girl human species done would be 10 minutes old the industrial area started 2 seconds again.another way to look at his wink of world GDP with the last 10000.it's a curious 8 for normal condition.what is the cost of this current analyse people would say it's technology now it's true Technology has a cumulated.and right now Technology advantages extremely that is the proximate cost that's why we are currently salary product.father to the ultimate cost look at this to highly distinguish we have.his master 200 lexical tokens and incredible seats and Edwin list the second super string Revolution if you are under the h.this is what we fine please invisible differences than.complicated 250 generation s

# Doing Abstractive Summarization

In [12]:
# Load tokenizer
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-wikihow")

In [13]:
# Load model
modelAbs = PegasusForConditionalGeneration.from_pretrained("google/pegasus-wikihow")

In [14]:
# Return the summary of the input Text
def GenerateSummary(text, x = 0.25):

    listTemp = []
    main_listText = []
    
    inputList = list(text.split('.'))
    l = len(inputList)
    chunk_size = math.ceil(l * x)      
    i = 0
    j = chunk_size - 1
    iterations = l / (chunk_size)

    while i < l:
        listTemp.append(inputList[i])
        if len(listTemp) >= chunk_size:
            main_listText.append('.'.join(listTemp))
            listTemp = []
        i += 1
    if len(listTemp) != 0:
        main_listText.append('.'.join(listTemp))
    
    # Create tokens - number representation of our text
    token = tokenizer(main_listText, truncation = True, padding = "longest", return_tensors = "pt")
    # Summarize
    summary = modelAbs.generate(**token)
    i = 0
    summaryText = ""
    while(i < len(summary)):
        summaryText += tokenizer.decode(summary[i])
        i += 1
    return summaryText

In [15]:
summary=GenerateSummary(final_text, x = 0.25)
print(summary)

<pad>Look at the human condition.<n>Look at what is the cost of this current analyse people would say it's technology now it's true Technology has a cumulated.and right now Technology advantages extremely that is the proximate cost that's why we are currently salary product.father to the ultimate cost look at this to highly distinguish we have.his master 200 lexical tokens and incredible seats and Edwin list the second super string Revolution if you are under the h.this is what we fine please invisible differences than.</s><pad>Take a look at how far away we are from being able to match a matchstick.<n>See how far away we are from being able to think like a human being.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [16]:
import re
clean_text=re.sub('<.*?>','',summary)
clean_text

"Look at the human condition.Look at what is the cost of this current analyse people would say it's technology now it's true Technology has a cumulated.and right now Technology advantages extremely that is the proximate cost that's why we are currently salary product.father to the ultimate cost look at this to highly distinguish we have.his master 200 lexical tokens and incredible seats and Edwin list the second super string Revolution if you are under the h.this is what we fine please invisible differences than.Take a look at how far away we are from being able to match a matchstick.See how far away we are from being able to think like a human being.First of all we need to understand what it means to be in the future.Here is a list of some of the things that we need to think about.First of all I would like to say thank you for reading this article and also for taking the time to read it and also for sharing your thoughts with me."